In [1]:
import geopandas as gpd
import h3pandas

In [2]:
pois = gpd.read_file('/Users/fabion/Downloads/arizona-latest-free.shp/gis_osm_pois_free_1.shp')

In [3]:
pois['fclass'].unique()

array(['viewpoint', 'tourist_info', 'archaeological', 'caravan_site',
       'shelter', 'camp_site', 'convenience', 'attraction', 'ruins',
       'supermarket', 'car_wash', 'water_well', 'restaurant', 'fast_food',
       'market_place', 'college', 'post_box', 'picnic_site', 'toilet',
       'butcher', 'fire_station', 'doityourself', 'hotel', 'post_office',
       'police', 'courthouse', 'library', 'tower', 'cafe', 'pub', 'bank',
       'atm', 'bar', 'water_tower', 'monument', 'park', 'school',
       'hospital', 'museum', 'memorial', 'observation_tower',
       'community_centre', 'doctors', 'nursing_home', 'comms_tower',
       'prison', 'public_building', 'embassy', 'sports_centre', 'theatre',
       'kindergarten', 'arts_centre', 'town_hall', 'water_works',
       'bookshop', 'nightclub', 'vending_any', 'cinema', 'artwork',
       'pharmacy', 'gift_shop', 'department_store', 'drinking_water',
       'university', 'hairdresser', 'fountain', 'computer_shop',
       'bicycle_rental', '

In [5]:
pois = pois.h3.geo_to_h3(8)

In [7]:
pois

osm_id  code               fclass  \
h3_08                                                    
8829b6d1a1fffff    42437147  2742            viewpoint   
8848e96921fffff   148874205  2701         tourist_info   
8848ebb759fffff   150932314  2733       archaeological   
8829b45981fffff   150938358  2424         caravan_site   
8829b6d009fffff   150941034  2424         caravan_site   
...                     ...   ...                  ...   
8829b6d32bfffff  9657414911  2542         bicycle_shop   
8829b6d32bfffff  9657414912  2305                  bar   
8848e84f47fffff  9657511965  2907  camera_surveillance   
8829b6d963fffff  9658540276  2561          hairdresser   
8848eba30dfffff  9658540277  2906         waste_basket   

                                    name                     geometry  
h3_08                                                                  
8829b6d1a1fffff                     None  POINT (-112.04591 33.56895)  
8848e96921fffff                     None  POINT (-112.00502 33.36145)  
8848ebb759fffff                Snaketown  POINT (-111.91958 33.18311)  
8829b45981fffff  Saddle Mountain RV Park  POINT (-112.93494 33.48785)  
8829b6d009fffff     A and F Trailer Park  POINT (-112.11098 33.49449)  
...                                  ...                          ...  
8829b6d32bfffff   Brass Monkey Bike Shop  POINT (-112.04485 33.46597)  
8829b6d32bfffff        Karamba Nightclub  POINT (-112.04463 33.46596)  
8848e84f47fffff                     None  POINT (-110.71422 32.41541)  
8829b6d963fffff              Great Clips  POINT (-111.83464 33.58850)  
8848eba30dfffff                     None  POINT (-111.70719 33.27922)  

[36822 rows x 5 columns]

## Connect to Solution

In [9]:
import pyTigerGraph as tg
import json

hostName = "https://globallookup.i.tgcloud.io"
userName = "tigergraph"
password = "test123"

conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password)

print("Connected")

Connected


# Create Schema

## Create Vertices

In [20]:
conn.gsql(f'''
CREATE VERTEX viewpoint(PRIMARY_ID id INT, name STRING, h3_8 STRING, osm_id INT, geometry STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
CREATE VERTEX h3(PRIMARY_ID id INT, h3_str STRING, res INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"
''')

'Successfully created vertex types: [viewpoint].\nSuccessfully created vertex types: [h3].'

## Create Edges

In [23]:
conn.gsql('''
CREATE UNDIRECTED EDGE in_h3(FROM viewpoint, TO h3)
''')

'Successfully created edge types: [in_h3].'

## Create Graph

In [24]:
conn.gsql('''
    CREATE GRAPH OSM_POI(viewpoint, h3, in_h3)
''')

'Stopping GPE GSE RESTPP\nSuccessfully stopped GPE GSE RESTPP in 25.946 seconds\nStarting GPE GSE RESTPP\nSuccessfully started GPE GSE RESTPP in 0.077 seconds\nThe graph OSM_POI is created.'

# Connect to Graph

In [25]:
graphName = "OSM_POI"
conn.graphname = graphName
secret = conn.createSecret()
token = conn.getToken(secret, setToken=True)[0]

print(token)

# connect to graph with token
conn = tg.TigerGraphConnection(host=hostName, username=userName, password=password, graphname=graphName, apiToken=token)

mk11k24f0i91579fkovikhiip7kiac49


# Load Data

## Create Loading Jobs

In [4]:
results = conn.gsql('''
USE GRAPH Northwind
BEGIN
CREATE LOADING JOB load_job_categories FOR GRAPH Northwind {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Category VALUES($0, $1, $2) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
    }
END
''')

In [5]:
results = conn.gsql('''
USE GRAPH Northwind
BEGIN
CREATE LOADING JOB load_job_customers FOR GRAPH Northwind {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Customer VALUES($0, $2, $3) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_phone VALUES($0 Customer, $9 Phone) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE at_address VALUES($0 Customer, $4 Address) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_region VALUES($0 Customer, $6 Region) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_country VALUES($0 Customer, $8 Country) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_postal_code VALUES($0 Customer, $7 PostalCode) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_city VALUES($0 Customer, $5 City) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_fax VALUES($0 Customer, $10 Fax) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
    }
END
''')

In [6]:
results = conn.gsql('''
USE GRAPH Northwind
BEGIN
CREATE LOADING JOB load_job_employee_territories FOR GRAPH Northwind {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE in_territory VALUES($0, $1) USING SEPARATOR="|", HEADER="true", EOL="\n";
    }
END
''')

In [7]:
results = conn.gsql('''
USE GRAPH Northwind
BEGIN
CREATE LOADING JOB load_job_order_details FOR GRAPH Northwind {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE contains_product VALUES($0, $1, $2, $3, $4) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
    }
END
''')

In [8]:
results = conn.gsql('''
USE GRAPH Northwind
BEGIN
CREATE LOADING JOB load_job_employees FOR GRAPH Northwind {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Employee VALUES($0, $1, $2, $5, $6, $17, _, $15) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE reports_to VALUES($0, $16) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_toc VALUES($0, $4) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_title VALUES($0, $3) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_phone VALUES($0 Employee, $12 Phone) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE at_address VALUES($0 Employee, $7 Address) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_region VALUES($0 Employee, $9 Region) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_country VALUES($0 Employee, $11 Country) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_postal_code VALUES($0 Employee, $10 PostalCode) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_city VALUES($0 Employee, $8 City) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
    }
END
''')

In [9]:
results = conn.gsql('''
USE GRAPH Northwind
BEGIN
CREATE LOADING JOB load_job_orders FOR GRAPH Northwind {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX CustomerOrder VALUES($0, $3, $4, $5, $6, $7, $8) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE placed_order VALUES($1, $0, $3) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE ship_via VALUES($0, $6) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE to_city VALUES($0, $10) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE to_postal_code VALUES($0, $12) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE to_country VALUES($0, $13) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE to_region VALUES($0, $11) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE to_address VALUES($0, $9) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE facilitated_order VALUES($0, $2) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
    }
END
''')

In [10]:
results = conn.gsql('''
USE GRAPH Northwind
BEGIN
CREATE LOADING JOB load_job_products FOR GRAPH Northwind {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Product VALUES($0, $1, $4, $5, $6, $7, $8, $9) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE is_category VALUES($0, $3) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE from_supplier VALUES($0, $2) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
    }
END
''')

In [11]:
results = conn.gsql('''
USE GRAPH Northwind
BEGIN
CREATE LOADING JOB load_job_regions FOR GRAPH Northwind {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Region VALUES($0, $1) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
    }
END
''')

In [12]:
results = conn.gsql('''
USE GRAPH Northwind
BEGIN
CREATE LOADING JOB load_job_shippers FOR GRAPH Northwind {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE from_company VALUES($0 Shipper, $1 Company) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_phone VALUES($0 Shipper, $2 Phone) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
    }
END
''')

In [13]:
results = conn.gsql('''
USE GRAPH Northwind
BEGIN
CREATE LOADING JOB load_job_suppliers FOR GRAPH Northwind {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Supplier VALUES($0, $2, $3, $11) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_fax VALUES($0 Supplier, $10 Fax) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_city VALUES($0 Supplier, $5 City) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_postal_code VALUES($0 Supplier, $7 PostalCode) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_country VALUES($0 Supplier, $8 Country) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_region VALUES($0 Supplier, $6 Region) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE at_address VALUES($0 Supplier, $4 Address) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE has_phone VALUES($0 Supplier, $9 Phone) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE from_company VALUES($0 Supplier, $1 Company) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
    }
END
''')

In [14]:
results = conn.gsql('''
USE GRAPH Northwind
BEGIN
CREATE LOADING JOB load_job_territories FOR GRAPH Northwind {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Territory VALUES($0, $1) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
      LOAD MyDataSource TO EDGE in_region VALUES($0 Territory, $2 Region) USING SEPARATOR="|", HEADER="true", EOL="\n", QUOTE="double";
    }
END
''')

## Running Loading Jobs

### Each Loading Job

In [ ]:
categories_file = "../data/categories.csv"
res = conn.uploadFile(categories_file, fileTag='MyDataSource', jobName='load_job_categories', sep="|")
print(json.dumps(res, indent=2))

In [ ]:
customers_file = "../data/customers.csv"
res = conn.uploadFile(customers_file, fileTag='MyDataSource', jobName='load_job_customers', sep="|")
print(json.dumps(res, indent=2))

In [ ]:
employee_territories_file = "../data/employee_territories.csv"
res = conn.uploadFile(employee_territories_file, fileTag='MyDataSource', jobName='load_job_employee_territories', sep="|")
print(json.dumps(res, indent=2))

In [ ]:
order_details_file = "../data/order_details.csv"
res = conn.uploadFile(order_details_file, fileTag='MyDataSource', jobName='load_job_order_details', sep="|")
print(json.dumps(res, indent=2))

In [ ]:
employees_file = "../data/employees.csv"
res = conn.uploadFile(employees_file, fileTag='MyDataSource', jobName='load_job_employees', sep="|")
print(json.dumps(res, indent=2))

In [ ]:
orders_file = "../data/orders.csv"
res = conn.uploadFile(orders_file, fileTag='MyDataSource', jobName='load_job_orders', sep="|")
print(json.dumps(res, indent=2))

In [ ]:
products_file = "../data/products.csv"
res = conn.uploadFile(products_file, fileTag='MyDataSource', jobName='load_job_products', sep="|")
print(json.dumps(res, indent=2))

In [ ]:
regions_file = "../data/regions.csv"
res = conn.uploadFile(regions_file, fileTag='MyDataSource', jobName='load_job_regions', sep="|")
print(json.dumps(res, indent=2))

In [ ]:
shippers_file = "../data/shippers.csv"
res = conn.uploadFile(shippers_file, fileTag='MyDataSource', jobName='load_job_shippers', sep="|")
print(json.dumps(res, indent=2))

In [ ]:
suppliers_file = '../data/suppliers.csv'
res = conn.uploadFile(suppliers_file, fileTag='MyDataSource', jobName='load_job_suppliers', sep="|")
print(json.dumps(res, indent=2))

In [ ]:
territories_file = "../data/territories.csv"
res = conn.uploadFile(territories_file, fileTag='MyDataSource', jobName='load_job_territories', sep="|")
print(json.dumps(res, indent=2))

# Running Queries

## Interpreting Queries

### Subtotal of an Order
`Price * Quantity * (1 - Discount)` for each item in the given order.

In [7]:
conn.runInterpretedQuery("""
INTERPRET QUERY get_order_subtotals(VERTEX<CustomerOrder> inOrder) FOR GRAPH Northwind { 
  
  SumAccum<FLOAT> @subtotal;
  
  thisOrder = {inOrder};
  
  results = SELECT o FROM thisOrder:o - (contains_product:e) - Product
    ACCUM
      o.@subtotal += e.unitPrice * e.quantity * (1.0 - e.discount);
  
  PRINT results [results.@subtotal] AS results;
}
""", params={ "inOrder": "11042" }
)

[{'results': [{'v_id': '11042',
    'v_type': 'CustomerOrder',
    'attributes': {'results.@subtotal': 405.75}}]}]

### Sales by Year
Get all sales subtotals within a given year.

In [26]:
conn.runInterpretedQuery("""
INTERPRET QUERY get_sales_by_year(int in_year) FOR GRAPH Northwind { 

  SumAccum<FLOAT> @subtotal;
  
  cust_orders = {CustomerOrder.*};
  
  results = SELECT o FROM cust_orders:o - (contains_product:e) - Product
    WHERE
      year(o.shippedDate) == in_year
    ACCUM
      o.@subtotal += e.unitPrice * e.quantity * (1.0 - e.discount)
    ORDER BY
      o.shippedDate;
  
  PRINT results [results.shippedDate, results.@subtotal, year(results.shippedDate)];
}
""", params={ "inYear": "1997" }
)

[{'results': [{'v_id': '10392',
    'v_type': 'CustomerOrder',
    'attributes': {'results.shippedDate': '1997-01-01 00:00:00',
     'results.@subtotal': 1440,
     'year(results.shippedDate)': 1997}},
   {'v_id': '10397',
    'v_type': 'CustomerOrder',
    'attributes': {'results.shippedDate': '1997-01-02 00:00:00',
     'results.@subtotal': 716.72003,
     'year(results.shippedDate)': 1997}},
   {'v_id': '10393',
    'v_type': 'CustomerOrder',
    'attributes': {'results.shippedDate': '1997-01-03 00:00:00',
     'results.@subtotal': 2556.94995,
     'year(results.shippedDate)': 1997}},
   {'v_id': '10395',
    'v_type': 'CustomerOrder',
    'attributes': {'results.shippedDate': '1997-01-03 00:00:00',
     'results.@subtotal': 2122.91992,
     'year(results.shippedDate)': 1997}},
   {'v_id': '10394',
    'v_type': 'CustomerOrder',
    'attributes': {'results.shippedDate': '1997-01-03 00:00:00',
     'results.@subtotal': 442,
     'year(results.shippedDate)': 1997}},
   {'v_id': '10396

### Employee Sales by Country
Get all employees sales subtotal in a given Country.

In [28]:
conn.runInterpretedQuery("""
INTERPRET QUERY get_employee_sales_by_country(VERTEX<Country> in_cont) FOR GRAPH Northwind SYNTAX v2{ 
  
  country = {in_cont};
  
  SumAccum<FLOAT> @subtotal;
  
  orders = SELECT o FROM country:c - (to_country) - CustomerOrder:o;
  
  subs = SELECT o FROM orders:o - (contains_product:e) - Product
    ACCUM
      o.@subtotal += e.unitPrice * e.quantity * (1.0 - e.discount);
    
  results = SELECT e FROM country:c - (to_country) - CustomerOrder:o - (facilitated_order) - Employee:e
    ACCUM
      e.@subtotal += o.@subtotal;
  
  PRINT country;
  PRINT results [results.firstName, results.lastName, results.@subtotal];
  
}
""", params={ "in_cont": "USA" }
)

[{'country': [{'v_id': 'USA',
    'v_type': 'Country',
    'attributes': {'id': '', '@subtotal': 0}}]},
 {'results': [{'v_id': '7',
    'v_type': 'Employee',
    'attributes': {'results.firstName': 'Robert',
     'results.lastName': 'King',
     'results.@subtotal': 23850.44922}},
   {'v_id': '8',
    'v_type': 'Employee',
    'attributes': {'results.firstName': 'Laura',
     'results.lastName': 'Callahan',
     'results.@subtotal': 25755.48438}},
   {'v_id': '4',
    'v_type': 'Employee',
    'attributes': {'results.firstName': 'Margaret',
     'results.lastName': 'Peacock',
     'results.@subtotal': 46384.83594}},
   {'v_id': '2',
    'v_type': 'Employee',
    'attributes': {'results.firstName': 'Andrew',
     'results.lastName': 'Fuller',
     'results.@subtotal': 22054.34961}},
   {'v_id': '5',
    'v_type': 'Employee',
    'attributes': {'results.firstName': 'Steven',
     'results.lastName': 'Buchanan',
     'results.@subtotal': 15172.05078}},
   {'v_id': '3',
    'v_type': 'Empl

### Alphabetical Products
List all products in alphabetical order.

In [29]:
conn.runInterpretedQuery("""
INTERPRET QUERY alphabetical_products() FOR GRAPH Northwind { 
  products = {Product.*};
  
  results = SELECT p FROM products:p
    ORDER BY
      p.productName ASC;
  
  PRINT results;
}
""")

[{'results': [{'v_id': 'ProductID',
    'v_type': 'Product',
    'attributes': {'id': 0,
     'productName': '',
     'qtyPerUnit': '',
     'unitPrice': 0,
     'inStock': 0,
     'onOrder': 0,
     'reorderLevel': 0,
     'discontinued': False}},
   {'v_id': '17',
    'v_type': 'Product',
    'attributes': {'id': 17,
     'productName': 'Alice Mutton',
     'qtyPerUnit': '20 - 1 kg tins',
     'unitPrice': 39,
     'inStock': 0,
     'onOrder': 0,
     'reorderLevel': 0,
     'discontinued': True}},
   {'v_id': '3',
    'v_type': 'Product',
    'attributes': {'id': 3,
     'productName': 'Aniseed Syrup',
     'qtyPerUnit': '12 - 550 ml bottles',
     'unitPrice': 10,
     'inStock': 13,
     'onOrder': 70,
     'reorderLevel': 25,
     'discontinued': False}},
   {'v_id': '40',
    'v_type': 'Product',
    'attributes': {'id': 40,
     'productName': 'Boston Crab Meat',
     'qtyPerUnit': '24 - 4 oz tins',
     'unitPrice': 18.4,
     'inStock': 123,
     'onOrder': 0,
     'reorderL

### Current Products
Get all products that aren't discontinued.

In [30]:
conn.runInterpretedQuery("""
INTERPRET QUERY get_current_product_list() FOR GRAPH Northwind { 
  
  products = {Product.*};
  
  results = SELECT p FROM products:p
    WHERE
      p.discontinued == FALSE
    ORDER BY
      p.productName ASC;
  
  PRINT results [results.id, results.productName];
}
""")

[{'results': [{'v_id': 'ProductID',
    'v_type': 'Product',
    'attributes': {'results.id': 0, 'results.productName': ''}},
   {'v_id': '3',
    'v_type': 'Product',
    'attributes': {'results.id': 3, 'results.productName': 'Aniseed Syrup'}},
   {'v_id': '40',
    'v_type': 'Product',
    'attributes': {'results.id': 40,
     'results.productName': 'Boston Crab Meat'}},
   {'v_id': '60',
    'v_type': 'Product',
    'attributes': {'results.id': 60,
     'results.productName': 'Camembert Pierrot'}},
   {'v_id': '18',
    'v_type': 'Product',
    'attributes': {'results.id': 18,
     'results.productName': 'Carnarvon Tigers'}},
   {'v_id': '1',
    'v_type': 'Product',
    'attributes': {'results.id': 1, 'results.productName': 'Chai'}},
   {'v_id': '2',
    'v_type': 'Product',
    'attributes': {'results.id': 2, 'results.productName': 'Chang'}},
   {'v_id': '39',
    'v_type': 'Product',
    'attributes': {'results.id': 39,
     'results.productName': 'Chartreuse verte'}},
   {'v_id'

### Extended Order Details
Get the details of each product for each order.

In [31]:
conn.runInterpretedQuery("""
INTERPRET QUERY get_order_details_extended() FOR GRAPH Northwind { 
  TYPEDEF TUPLE <INT product_id, STRING product_name, FLOAT unit_price, INT qty, FLOAT discnt, FLOAT ext_price> product_details;
  c_order = {CustomerOrder.*};
  
  MapAccum<VERTEX<Product>, product_details> @extended_price;
  
  results = SELECT o FROM c_order:o - (:e) - Product:p
    ACCUM
      FLOAT extended_price = e.unitPrice * e.quantity * (1 - e.discount),
      o.@extended_price += (p -> product_details(p.id, p.productName, e.unitPrice, e.quantity, e.discount, extended_price))
    ORDER BY
      o.id ASC;
  
  PRINT results [results.id, results.@extended_price];
}
""")

[{'results': [{'v_id': '11074',
    'v_type': 'CustomerOrder',
    'attributes': {'results.id': 0,
     'results.@extended_price': {'16': {'product_id': 16,
       'product_name': 'Pavlova',
       'unit_price': 17.45,
       'qty': 14,
       'discnt': 0.05,
       'ext_price': 232.08502}}}},
   {'v_id': '11062',
    'v_type': 'CustomerOrder',
    'attributes': {'results.id': 0,
     'results.@extended_price': {'53': {'product_id': 53,
       'product_name': 'Perth Pasties',
       'unit_price': 32.8,
       'qty': 10,
       'discnt': 0.2,
       'ext_price': 262.39999},
      '70': {'product_id': 70,
       'product_name': 'Outback Lager',
       'unit_price': 15,
       'qty': 12,
       'discnt': 0.2,
       'ext_price': 144}}}},
   {'v_id': '11077',
    'v_type': 'CustomerOrder',
    'attributes': {'results.id': 0,
     'results.@extended_price': {'20': {'product_id': 20,
       'product_name': "Sir Rodney's Marmalade",
       'unit_price': 81,
       'qty': 1,
       'discnt': 0